# 1. Import Libraries 

In [47]:
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from rdflib.util import guess_format

from SPARQLWrapper import SPARQLWrapper, JSON, N3

import pandas as pd
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

#Used from INM713 Semantic Web Technologies and Knowledge Graphs Lab 5
from isub import isub
from lookup import DBpediaLookup

import csv
import owlrl

import math

# 2. Import Data

### 2.1 Import class attributes and save as list

In [48]:
predicates_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/predicates.txt", sep = '\t',header = None, names = ['predicates'])
predicate_list = predicates_df['predicates'].tolist()
print(predicate_list)

['black', 'white', 'blue', 'brown', 'gray', 'orange', 'red', 'yellow', 'patches', 'spots', 'stripes', 'furry', 'hairless', 'toughskin', 'big', 'small', 'bulbous', 'lean', 'flippers', 'hands', 'hooves', 'pads', 'paws', 'longleg', 'longneck', 'tail', 'chewteeth', 'meatteeth', 'buckteeth', 'strainteeth', 'horns', 'claws', 'tusks', 'smelly', 'flys', 'hops', 'swims', 'tunnels', 'walks', 'fast', 'slow', 'strong', 'weak', 'muscle', 'bipedal', 'quadrapedal', 'active', 'inactive', 'nocturnal', 'hibernate', 'agility', 'fish', 'meat', 'plankton', 'vegetation', 'insects', 'forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker', 'newworld', 'oldworld', 'arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean', 'ground', 'water', 'tree', 'cave', 'fierce', 'timid', 'smart', 'group', 'solitary', 'nestspot', 'domestic']


### 2.2. Import Predicate Matrix

### 2.2.1 Binary

In [49]:
predicateMatrixBinary_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/predicate-matrix-binary.txt", sep = " ",header = None, names = predicate_list)
predicateMatrixBinary_df.head()

,black,white,blue,brown,gray,orange,red,yellow,patches,spots,stripes,furry,hairless,toughskin,big,small,bulbous,lean,flippers,hands,hooves,pads,paws,longleg,longneck,tail,chewteeth,meatteeth,buckteeth,strainteeth,horns,claws,tusks,smelly,flys,hops,swims,tunnels,walks,fast,slow,strong,weak,muscle,bipedal,quadrapedal,active,inactive,nocturnal,hibernate,agility,fish,meat,plankton,vegetation,insects,forager,grazer,hunter,scavenger,skimmer,stalker,newworld,oldworld,arctic,coastal,desert,bush,plains,forest,fields,jungle,mountains,ocean,ground,water,tree,cave,fierce,timid,smart,group,solitary,nestspot,domestic
0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0
2,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0
4,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1


### 2.3 Load csv with animal classes

In [50]:
classes_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/classes.txt", sep = '\t',header = None, names = ['labels','classes'])
classes_df.head()

,labels,classes
0,1,antelope
1,2,grizzly+bear
2,3,killer+whale
3,4,beaver
4,5,dalmatian


### 2.4 Rename animal classes

In [51]:
classes_list = ["Antelope","Grizzly bear","Killer Whale","Beaver", "Dalmatian (dog)", "Persian cat", "Horse", "German Shepherd", 
               "Blue whale", "Siamese cat", "Skunk", "Mole (animal)", "Tiger", "Hippopotamus", "Leopard", "Moose", 
               "Spider monkey", "Humpback whale", "Elephant", "Gorilla", "Ox", "Fox", "Sheep", "Seal", "Chimpanzee", 
                "Hamster", "Squirrel", "Rhinoceros","Rabbit", "Bat", "Giraffe", "Wolf", "Chihuahua (dog)",
               "Rat", "Weasel", "Otter", "Buffalo", "Zebra", "Giant panda","Deer", "Bobcat", "Pig", "Lion", "Mouse",
                "Polar bear", "Collie", "Walrus", "Raccoon", "Cattle", "Dolphin"]

### 2.5  Concat Class labels and predicate matrix

In [52]:
predicate_df = pd.concat([classes_df, predicateMatrixBinary_df], axis=1)
predicate_df['corrected_class_names'] = classes_list
predicate_df

,labels,classes,black,white,blue,brown,gray,orange,red,yellow,patches,spots,stripes,furry,hairless,toughskin,big,small,bulbous,lean,flippers,hands,hooves,pads,paws,longleg,longneck,tail,chewteeth,meatteeth,buckteeth,strainteeth,horns,claws,tusks,smelly,flys,hops,swims,tunnels,walks,fast,slow,strong,weak,muscle,bipedal,quadrapedal,active,inactive,nocturnal,hibernate,agility,fish,meat,plankton,vegetation,insects,forager,grazer,hunter,scavenger,skimmer,stalker,newworld,oldworld,arctic,coastal,desert,bush,plains,forest,fields,jungle,mountains,ocean,ground,water,tree,cave,fierce,timid,smart,group,solitary,nestspot,domestic,corrected_class_names
0,1,antelope,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,Antelope
1,2,grizzly+bear,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0,Grizzly bear
2,3,killer+whale,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,Killer Whale
3,4,beaver,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0,Beaver
4,5,dalmatian,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1,Dalmatian (dog)
5,6,persian+cat,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,Persian cat
6,7,horse,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,1,1,0,0,1,Horse
7,8,german+shepherd,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,German Shepherd
8,9,blue+whale,0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,Blue whale
9,10,siamese+cat,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,1,Siamese cat


### 2.6 Store attributes/column names as a list

In [53]:
attributes = []
for col in predicate_df.columns:
    attributes.append(col)
attributes.remove('labels')
attributes.remove('classes')
attributes.remove('corrected_class_names')

In [54]:
print(attributes)

['black', 'white', 'blue', 'brown', 'gray', 'orange', 'red', 'yellow', 'patches', 'spots', 'stripes', 'furry', 'hairless', 'toughskin', 'big', 'small', 'bulbous', 'lean', 'flippers', 'hands', 'hooves', 'pads', 'paws', 'longleg', 'longneck', 'tail', 'chewteeth', 'meatteeth', 'buckteeth', 'strainteeth', 'horns', 'claws', 'tusks', 'smelly', 'flys', 'hops', 'swims', 'tunnels', 'walks', 'fast', 'slow', 'strong', 'weak', 'muscle', 'bipedal', 'quadrapedal', 'active', 'inactive', 'nocturnal', 'hibernate', 'agility', 'fish', 'meat', 'plankton', 'vegetation', 'insects', 'forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker', 'newworld', 'oldworld', 'arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean', 'ground', 'water', 'tree', 'cave', 'fierce', 'timid', 'smart', 'group', 'solitary', 'nestspot', 'domestic']


# 3. URI generation & RDF generation

### 3.1 Set up dictionary to hold the URI's
Adapted from N3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5 solution

In [55]:
#Dictionary that keeps the URIs. Specially useful if accessing a remote service to get a candidate URI to avoid repeated calls
stringToURI = dict()

## 3.2 Set up empty graph and Namespace

In [56]:
#Create empty graph
g = Graph()

#Define Namespace
awa2_onto_uri = "http://www.semanticweb.org/inm363/AwA2#"
      
AwA2 = Namespace(awa2_onto_uri)
dbo = Namespace("https://dbpedia.org/ontology/") 
owl = Namespace("http://www.w3.org/2002/07/owl#") 

#Bind Prefixes 
g.bind("AwA2", AwA2) #cw is a newly created namespace  
g.bind("dbo", dbo) 
g.bind("owl", owl) 

## 3.3 Define Code to generate Triples
Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 3.3.1 Modular Mapping functions 

In [57]:
# Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

def is_nan( x):
        return (x != x)         
            
        
def createURIForEntity(name, useExternalURI):
    stringToURI[name] = AwA2 + name.replace(" ", "_")
    
    if useExternalURI is True: #We connect to online KG
        uri = getExternalKGURI(name)
        if uri!="":
            stringToURI[name]=uri
    
    return stringToURI[name]

  
def getExternalKGURI(name):
    entities = DBpediaLookup().getKGEntities(name, 2)
    current_sim = -1
    current_uri=''
    for ent in entities:           
        isub_score = isub(name, ent.label) 
        if current_sim < isub_score:
            current_uri = ent.ident
            current_sim = isub_score    
        
    return current_uri 

    
def mappingToCreateTypeTriple(dataframe, subject_column, class_type, useExternalURI):
        
        for subject in dataframe[subject_column]:
                
            if subject.lower() in stringToURI:
                entity_uri = stringToURI[subject.lower()]
            else:
                entity_uri = createURIForEntity(subject.lower(), useExternalURI)
            
            g.add((URIRef(entity_uri), RDF.type, class_type))
            print('Example genrated URI for',subject_column,' ',URIRef(entity_uri))
            print('Example generated rdf:type triple for',subject_column,' ',URIRef(entity_uri),RDF.type, class_type)
            print('')       


def mappingToCreateLiteralTriple(dataframe, subject_column, object_column, predicate, datatype):
        
        for subject, lit_value in zip(dataframe[subject_column], dataframe[object_column]):
            
            if is_nan(lit_value) or lit_value==None or lit_value=="":
                pass
            
            else:
                #Uri as already created
                entity_uri = stringToURI[subject.lower()]
                    
                #Literal
                lit = Literal(lit_value, datatype=datatype)
                
                #New triple
                g.add((URIRef(entity_uri), predicate, lit))                     
                                   
            print('Example generated data property triple for',subject_column,'and',object_column,':', URIRef(entity_uri), predicate, lit)
            print('')
                         
            
def mappingToCreateObjectTriple(dataframe, subject_column, object_name, predicate):
        
    for subject in dataframe[subject_column]:
            
        if is_nan(object):
            pass
            
        else:
            #Uri as already created
            subject_uri = stringToURI[subject.lower()]
            object_uri = stringToURI[object_name]
                    
            #New triple
            g.add((predicate, RDF.type, OWL.ObjectProperty))
            g.add((URIRef(subject_uri), predicate, URIRef(object_uri)))
        print('Object property triple',predicate, RDF.type, OWL.ObjectProperty)
        print('Example generated object property triple for',URIRef(subject_uri), predicate, URIRef(object_uri))
        print('')

### 3.4 Define Class Type Triples
Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 3.4.1 Animal classes

In [58]:
stringToURI.clear() #empty dict to remove all URIs from DBPedia
mappingToCreateTypeTriple(predicate_df,'classes', AwA2.Animal, False) #Comment to use user define URI

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#antelope
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363/AwA2#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#Animal

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#grizzly+bear
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#Animal

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#killer+whale
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#Animal

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#beaver
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363

### 3.4.2 Animal attributes

In [59]:
attributes_df = pd.DataFrame(attributes, columns = ['attributes'])
mappingToCreateTypeTriple(attributes_df,'attributes', AwA2.AnimalAttributes, False) 

Example genrated URI for attributes   http://www.semanticweb.org/inm363/AwA2#black
Example generated rdf:type triple for attributes   http://www.semanticweb.org/inm363/AwA2#black http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalAttributes

Example genrated URI for attributes   http://www.semanticweb.org/inm363/AwA2#white
Example generated rdf:type triple for attributes   http://www.semanticweb.org/inm363/AwA2#white http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalAttributes

Example genrated URI for attributes   http://www.semanticweb.org/inm363/AwA2#blue
Example generated rdf:type triple for attributes   http://www.semanticweb.org/inm363/AwA2#blue http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalAttributes

Example genrated URI for attributes   http://www.semanticweb.org/inm363/AwA2#brown
Example generated rdf:type triple for attributes   http://www.seman

### 3.5 Create Litral type Triples
Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

In [60]:
mappingToCreateLiteralTriple(predicate_df, 'classes', 'corrected_class_names',AwA2.animalNames, XSD.string)

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#animalNames Antelope

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#animalNames Grizzly bear

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#animalNames Killer Whale

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/inm363/AwA2#animalNames Beaver

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.semanticweb.org/inm363/AwA2#animalNames Dalmatian (dog)

Example generated data property triple f

### 3.5 Create Object Property Triples
Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 3.5.1 hasColour

In [61]:
colours = ['black','white','blue','brown','gray','orange','red','yellow']
for colour in colours:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[colour]==1],'classes', colour, AwA2.hasColour)


Object property triple http://www.semanticweb.org/inm363/AwA2#hasColour http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasColour http://www.semanticweb.org/inm363/AwA2#black

Object property triple http://www.semanticweb.org/inm363/AwA2#hasColour http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasColour http://www.semanticweb.org/inm363/AwA2#black

Object property triple http://www.semanticweb.org/inm363/AwA2#hasColour http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.semanticweb.org/

### 3.5.2 hasSkinType

In [62]:
skin_type = ['patches', 'spots', 'stripes', 'toughskin']
for skin in skin_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[skin]==1],'classes', skin, AwA2.hasSkinPattern)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasSkinPattern http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasSkinPattern http://www.semanticweb.org/inm363/AwA2#patches

Object property triple http://www.semanticweb.org/inm363/AwA2#hasSkinPattern http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.semanticweb.org/inm363/AwA2#hasSkinPattern http://www.semanticweb.org/inm363/AwA2#patches

Object property triple http://www.semanticweb.org/inm363/AwA2#hasSkinPattern http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#horse http:

### 3.5.3 hasFurType

In [63]:
fur_type = ['furry', 'hairless']
for fur in fur_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[fur]==1],'classes', fur, AwA2.hasFurType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFurType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasFurType http://www.semanticweb.org/inm363/AwA2#furry

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFurType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasFurType http://www.semanticweb.org/inm363/AwA2#furry

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFurType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/in

### 3.5.4 hasBodyType

In [64]:
body_type = ['big', 'small', 'bulbous', 'lean', 'muscle']
for body in body_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[body]==1],'classes', body, AwA2.hasBodyType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBodyType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasBodyType http://www.semanticweb.org/inm363/AwA2#big

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBodyType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasBodyType http://www.semanticweb.org/inm363/AwA2#big

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBodyType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb

### 3.5.5 hasAppendageType

In [65]:
appendage_type = ['flippers', 'hands', 'hooves', 'pads', 'paws']
for appendage in appendage_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[appendage]==1],'classes', appendage, AwA2.hasAppendageType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasAppendageType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasAppendageType http://www.semanticweb.org/inm363/AwA2#flippers

Object property triple http://www.semanticweb.org/inm363/AwA2#hasAppendageType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#blue+whale http://www.semanticweb.org/inm363/AwA2#hasAppendageType http://www.semanticweb.org/inm363/AwA2#flippers

Object property triple http://www.semanticweb.org/inm363/AwA2#hasAppendageType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA

### 3.5.6 hasTeethType

In [66]:
teeth_type = ['chewteeth', 'meatteeth', 'buckteeth', 'strainteeth']
for teeth in teeth_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[teeth]==1],'classes', teeth, AwA2.hasTeethType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.semanticweb.org/inm363/AwA2#chewteeth

Object property triple http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.semanticweb.org/inm363/AwA2#chewteeth

Object property triple http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.

### 3.5.7 hasMovementType

In [67]:
movement_type = ['flys', 'hops', 'swims', 'tunnels', 'walks']
for movement in movement_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[movement]==1],'classes', movement, AwA2.hasMovementType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasMovementType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#bat http://www.semanticweb.org/inm363/AwA2#hasMovementType http://www.semanticweb.org/inm363/AwA2#flys

Object property triple http://www.semanticweb.org/inm363/AwA2#hasMovementType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#hamster http://www.semanticweb.org/inm363/AwA2#hasMovementType http://www.semanticweb.org/inm363/AwA2#hops

Object property triple http://www.semanticweb.org/inm363/AwA2#hasMovementType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#squirrel http://www.sem

### 3.5.8 hasMovementSpeed

In [68]:
speed_type = ['fast','slow']
for speed in speed_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[speed]==1],'classes', speed, AwA2.hasMovementSpeed)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasMovementSpeed http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasMovementSpeed http://www.semanticweb.org/inm363/AwA2#fast

Object property triple http://www.semanticweb.org/inm363/AwA2#hasMovementSpeed http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasMovementSpeed http://www.semanticweb.org/inm363/AwA2#fast

Object property triple http://www.semanticweb.org/inm363/AwA2#hasMovementSpeed http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+w

### 3.5.9 hasStrengthType

In [69]:
strength_type = ['strong','weak']
for strength in strength_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[strength]==1],'classes', strength, AwA2.hasStrengthType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasStrengthType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasStrengthType http://www.semanticweb.org/inm363/AwA2#strong

Object property triple http://www.semanticweb.org/inm363/AwA2#hasStrengthType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasStrengthType http://www.semanticweb.org/inm363/AwA2#strong

Object property triple http://www.semanticweb.org/inm363/AwA2#hasStrengthType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+wh

### 3.5.10 hasNumLegs

In [70]:
num_legs = ['bipedal', 'quadrapedal']
for num in num_legs:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[num]==1],'classes', num, AwA2.hasNumLegs)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasNumLegs http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasNumLegs http://www.semanticweb.org/inm363/AwA2#bipedal

Object property triple http://www.semanticweb.org/inm363/AwA2#hasNumLegs http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#spider+monkey http://www.semanticweb.org/inm363/AwA2#hasNumLegs http://www.semanticweb.org/inm363/AwA2#bipedal

Object property triple http://www.semanticweb.org/inm363/AwA2#hasNumLegs http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#gorilla http://www.semantic

### 3.5.11 hasActivityType

In [71]:
activity_type = ['active', 'inactive']
for activity in activity_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[activity]==1],'classes', activity, AwA2.hasActivityType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasActivityType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasActivityType http://www.semanticweb.org/inm363/AwA2#active

Object property triple http://www.semanticweb.org/inm363/AwA2#hasActivityType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasActivityType http://www.semanticweb.org/inm363/AwA2#active

Object property triple http://www.semanticweb.org/inm363/AwA2#hasActivityType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+wh

### 3.5.12 hasActivityPeriod

In [72]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['nocturnal']==1],'classes', 'nocturnal', AwA2.hasActivityPeriod)


Object property triple http://www.semanticweb.org/inm363/AwA2#hasActivityPeriod http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasActivityPeriod http://www.semanticweb.org/inm363/AwA2#nocturnal

Object property triple http://www.semanticweb.org/inm363/AwA2#hasActivityPeriod http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/inm363/AwA2#hasActivityPeriod http://www.semanticweb.org/inm363/AwA2#nocturnal

Object property triple http://www.semanticweb.org/inm363/AwA2#hasActivityPeriod http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/

### 3.5.13 hasDietType

In [73]:
diet_type = ['fish', 'meat', 'plankton', 'vegetation', 'insects']
for diet in diet_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[diet]==1],'classes', diet, AwA2.hasDietType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasDietType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasDietType http://www.semanticweb.org/inm363/AwA2#fish

Object property triple http://www.semanticweb.org/inm363/AwA2#hasDietType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasDietType http://www.semanticweb.org/inm363/AwA2#fish

Object property triple http://www.semanticweb.org/inm363/AwA2#hasDietType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb

### 3.5.14 hasFoodCollectionType

In [74]:
food_collection_type = ['forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker']
for collection in food_collection_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[collection]==1],'classes', collection, AwA2.hasFoodCollectionType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.semanticweb.org/inm363/AwA2#forager

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.semanticweb.org/inm363/AwA2#forager

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.seman

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.semanticweb.org/inm363/AwA2#stalker

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#german+shepherd http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.semanticweb.org/inm363/AwA2#stalker

Object property triple http://www.semanticweb.org/inm363/AwA2#hasFoodCollectionType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://ww

### 3.5.15 fromWorldType

In [75]:
world_type = ['newworld','oldworld']
for world in world_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[world]==1],'classes', world, AwA2.fromWorldType)

Object property triple http://www.semanticweb.org/inm363/AwA2#fromWorldType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#fromWorldType http://www.semanticweb.org/inm363/AwA2#newworld

Object property triple http://www.semanticweb.org/inm363/AwA2#fromWorldType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#fromWorldType http://www.semanticweb.org/inm363/AwA2#newworld

Object property triple http://www.semanticweb.org/inm363/AwA2#fromWorldType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://w

### 3.5.16 hasHabitat

In [76]:
habitat_type = ['arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean']
for habitat in habitat_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[habitat]==1],'classes', habitat, AwA2.hasHabitat)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasHabitat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasHabitat http://www.semanticweb.org/inm363/AwA2#arctic

Object property triple http://www.semanticweb.org/inm363/AwA2#hasHabitat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#blue+whale http://www.semanticweb.org/inm363/AwA2#hasHabitat http://www.semanticweb.org/inm363/AwA2#arctic

Object property triple http://www.semanticweb.org/inm363/AwA2#hasHabitat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#moose http://www.semanticweb.org

### 3.5.17 livesIn

In [77]:
home_type = ['ground', 'water', 'tree', 'cave']
for home in home_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[home]==1],'classes', home, AwA2.livesIn)

Object property triple http://www.semanticweb.org/inm363/AwA2#livesIn http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#livesIn http://www.semanticweb.org/inm363/AwA2#ground

Object property triple http://www.semanticweb.org/inm363/AwA2#livesIn http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#livesIn http://www.semanticweb.org/inm363/AwA2#ground

Object property triple http://www.semanticweb.org/inm363/AwA2#livesIn http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/inm363/AwA2#liv

### 3.5.18 hasTemparment

In [78]:
temperment_type = ['fierce', 'timid']
for temperment in temperment_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[temperment]==1],'classes', temperment, AwA2.hasTemparment)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasTemparment http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasTemparment http://www.semanticweb.org/inm363/AwA2#fierce

Object property triple http://www.semanticweb.org/inm363/AwA2#hasTemparment http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasTemparment http://www.semanticweb.org/inm363/AwA2#fierce

Object property triple http://www.semanticweb.org/inm363/AwA2#hasTemparment http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#german+shepherd

### 3.5.19 hasGroupBehavior

In [79]:
group_type = ['group','solitary']
for group in group_type:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[group]==1],'classes', group, AwA2.hasGroupBehavior)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasGroupBehavior http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasGroupBehavior http://www.semanticweb.org/inm363/AwA2#group

Object property triple http://www.semanticweb.org/inm363/AwA2#hasGroupBehavior http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasGroupBehavior http://www.semanticweb.org/inm363/AwA2#group

Object property triple http://www.semanticweb.org/inm363/AwA2#hasGroupBehavior http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver

### 3.5.20 hasPhysicalUniqueFeature

In [80]:
unique_feature = ['longleg', 'longneck', 'horns', 'claws', 'tusks'] 
for feature in unique_feature:
    mappingToCreateObjectTriple(predicate_df.loc[predicate_df[feature]==1],'classes', feature, AwA2.hasPhysicalUniqueFeature)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasPhysicalUniqueFeature http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasPhysicalUniqueFeature http://www.semanticweb.org/inm363/AwA2#longleg

Object property triple http://www.semanticweb.org/inm363/AwA2#hasPhysicalUniqueFeature http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.semanticweb.org/inm363/AwA2#hasPhysicalUniqueFeature http://www.semanticweb.org/inm363/AwA2#longleg

Object property triple http://www.semanticweb.org/inm363/AwA2#hasPhysicalUniqueFeature http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http

### 3.5.21 hasBodySmell

In [81]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['smelly']==1],'classes', 'smelly', AwA2.hasBodySmell)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBodySmell http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#horse http://www.semanticweb.org/inm363/AwA2#hasBodySmell http://www.semanticweb.org/inm363/AwA2#smelly

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBodySmell http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#german+shepherd http://www.semanticweb.org/inm363/AwA2#hasBodySmell http://www.semanticweb.org/inm363/AwA2#smelly

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBodySmell http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#skunk http://www.semanti

### 3.5.22 hasIntelligence

In [82]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['smart']==1],'classes', 'smart', AwA2.hasIntelligence)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasIntelligence http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasIntelligence http://www.semanticweb.org/inm363/AwA2#smart

Object property triple http://www.semanticweb.org/inm363/AwA2#hasIntelligence http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasIntelligence http://www.semanticweb.org/inm363/AwA2#smart

Object property triple http://www.semanticweb.org/inm363/AwA2#hasIntelligence http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver 

### 3.5.23 hasLifeType

In [83]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['domestic']==1],'classes', 'domestic', AwA2.hasLifeType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasLifeType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.semanticweb.org/inm363/AwA2#hasLifeType http://www.semanticweb.org/inm363/AwA2#domestic

Object property triple http://www.semanticweb.org/inm363/AwA2#hasLifeType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#persian+cat http://www.semanticweb.org/inm363/AwA2#hasLifeType http://www.semanticweb.org/inm363/AwA2#domestic

Object property triple http://www.semanticweb.org/inm363/AwA2#hasLifeType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#horse http://www.semantic

### 3.5.24 hasChildCareType

In [84]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['nestspot']==1],'classes', 'nestspot', AwA2.hasChildCareType)

Object property triple http://www.semanticweb.org/inm363/AwA2#hasChildCareType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/inm363/AwA2#hasChildCareType http://www.semanticweb.org/inm363/AwA2#nestspot

Object property triple http://www.semanticweb.org/inm363/AwA2#hasChildCareType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#mole http://www.semanticweb.org/inm363/AwA2#hasChildCareType http://www.semanticweb.org/inm363/AwA2#nestspot

Object property triple http://www.semanticweb.org/inm363/AwA2#hasChildCareType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#tiger http

### 3.5.25 hasPhyscialCoordination

In [85]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['agility']==1],'classes', 'agility', AwA2.hasPhyscialCoordination)


Object property triple http://www.semanticweb.org/inm363/AwA2#hasPhyscialCoordination http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#hasPhyscialCoordination http://www.semanticweb.org/inm363/AwA2#agility

Object property triple http://www.semanticweb.org/inm363/AwA2#hasPhyscialCoordination http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#hasPhyscialCoordination http://www.semanticweb.org/inm363/AwA2#agility

Object property triple http://www.semanticweb.org/inm363/AwA2#hasPhyscialCoordination http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http:/

### 3.2.26 hasBehavioralAdaptation

In [86]:
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['hibernate']==1],'classes', 'hibernate', AwA2.hasBehavioralAdaptation)


Object property triple http://www.semanticweb.org/inm363/AwA2#hasBehavioralAdaptation http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#hasBehavioralAdaptation http://www.semanticweb.org/inm363/AwA2#hibernate

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBehavioralAdaptation http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/inm363/AwA2#hasBehavioralAdaptation http://www.semanticweb.org/inm363/AwA2#hibernate

Object property triple http://www.semanticweb.org/inm363/AwA2#hasBehavioralAdaptation http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
Example generated object property triple for http

## 4. Print all triples in the  generated RDF data

In [87]:
print("The graph contains '" + str(len(g)) + "' triples.")
for subj, pred, obj in g:
    print(subj, pred, obj)

The graph contains '1734' triples.
http://www.semanticweb.org/inm363/AwA2#hippopotamus http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.semanticweb.org/inm363/AwA2#strainteeth
http://www.semanticweb.org/inm363/AwA2#rhinoceros http://www.semanticweb.org/inm363/AwA2#hasMovementType http://www.semanticweb.org/inm363/AwA2#walks
http://www.semanticweb.org/inm363/AwA2#humpback+whale http://www.semanticweb.org/inm363/AwA2#hasIntelligence http://www.semanticweb.org/inm363/AwA2#smart
http://www.semanticweb.org/inm363/AwA2#weasel http://www.semanticweb.org/inm363/AwA2#hasTeethType http://www.semanticweb.org/inm363/AwA2#meatteeth
http://www.semanticweb.org/inm363/AwA2#bat http://www.semanticweb.org/inm363/AwA2#fromWorldType http://www.semanticweb.org/inm363/AwA2#oldworld
http://www.semanticweb.org/inm363/AwA2#bat http://www.semanticweb.org/inm363/AwA2#hasColour http://www.semanticweb.org/inm363/AwA2#black
http://www.semanticweb.org/inm363/AwA2#fox http://www.semanticweb.org/inm363/A

## 5. Perform Reasoning

In [88]:
#print("Data triples before reasoning: '" + str(len(g)) + "'.")

#g_expanded = g

##Perform reasoning
#owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=True, datatype_axioms=False).expand(g_expanded)
#print("Data triples after reasoning: '" + str(len(g_expanded)) + "'.")

In [89]:
#print("The graph contains '" + str(len(g_expanded)) + "' triples.")
#for subj, pred, obj in g_expanded:
    #print(subj, pred, obj)

## 6. Save the Generated RDF data in turtle format (.ttl)

In [90]:
g.serialize(destination='AWA2_KG01_basic_attributes_01.ttl', format='ttl') 
g.serialize(destination='C:/Users/Abhimanyu Acharya/Documents/GitHub/Dissertation/OWL2Vec-Star-master/dissertation/AWA2_KG01_basic_attributes_01.owl', format='xml') 


## 7. Run some queries to check the generated RDF data

In [91]:
g_test = Graph()

#Load Generated RDF
Generated_rdf = "AWA2_KG01_basic_attributes_01.ttl" # define path to Generated RDF
g_test.load(Generated_rdf ,  format=guess_format(Generated_rdf)) #load Generated RDF

In [92]:
qres = g_test.query("""
                SELECT DISTINCT ?animals WHERE {
                ?animals rdf:type AwA2:Animal .  
                ?animals AwA2:hasColour AwA2:white . 
                ?animals AwA2:hasGroupBehavior AwA2:group .}    
            """)


print("%s animals satisfying the query." % (str(len(qres))))

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    line_str = row.animals
    print(line_str)

14 animals satisfying the query.
http://www.semanticweb.org/inm363/AwA2#wolf
http://www.semanticweb.org/inm363/AwA2#sheep
http://www.semanticweb.org/inm363/AwA2#pig
http://www.semanticweb.org/inm363/AwA2#giant+panda
http://www.semanticweb.org/inm363/AwA2#dalmatian
http://www.semanticweb.org/inm363/AwA2#tiger
http://www.semanticweb.org/inm363/AwA2#horse
http://www.semanticweb.org/inm363/AwA2#dolphin
http://www.semanticweb.org/inm363/AwA2#killer+whale
http://www.semanticweb.org/inm363/AwA2#zebra
http://www.semanticweb.org/inm363/AwA2#rabbit
http://www.semanticweb.org/inm363/AwA2#cow
http://www.semanticweb.org/inm363/AwA2#seal
http://www.semanticweb.org/inm363/AwA2#mouse
